In [1]:
from decoder_new import *
from encoder import *
from data_loader import *
import pickle
import random
import torch.optim as optim
from torch.autograd import Variable
import csv
import time
from tqdm import tqdm
import gc
import os
import torchvision.transforms as tf

[nltk_data] Downloading package punkt to
[nltk_data]     /datasets/home/64/364/rhadden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def trainEncoderDecoder(encoder, decoder, criterion, epochs,
                        train_loader,val_loader, test_loader, name, batch_size, maxSeqLen, vocablen):
    
    #Create non-existing logfiles
    logname = './logs/' + name + '.log'
    i = 0
    if os.path.exists(logname) == True:
        
        logname = './logs/' + name + str(i) + '.log'
        while os.path.exists(logname):
            i+=1
            logname = './logs/' + name + str(i) + '.log'

    print('Loading results to logfile: ' + logname)
    with open(logname, "w") as file:
        file.write("Log file DATA: Validation Loss and Accuracy\n") 
    
    logname_summary = './logs/' + name + '_summary' + str(i) + '.log'    
    print('Loading Summary to : ' + logname_summary) 
    
    parameters = list(encoder.parameters())
    parameters.extend(list(decoder.parameters()))
    optimizer = optim.Adam(parameters, lr=5e-3)
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        device = torch.device("cuda:0")
#         encoder = torch.nn.DataParallel(encoder)
#         decoder = torch.nn.DataParallel(decoder)
        
        encoder.to(device)
        decoder.to(device)
        
        
    
    val_loss_set = []
    val_acc_set = []
    val_iou_set = []
    
    
    training_loss = []
    
    # Early Stop criteria
    minLoss = 1e6
    minLossIdx = 0
    earliestStopEpoch = 10
    earlyStopDelta = 5
    for epoch in range(epochs):
        ts = time.time()

        #import pdb; pdb.set_trace()                     
        for iter, (inputs, labels, lengths, actual_lengths) in tqdm(enumerate(train_loader)):
            print('inp shape: ', inputs.shape)
#             print("Inputs:")
#             print(inputs)
            
            print("Labels")
            print(labels)
            print('labels shape: ', labels.shape)
            print("lengths: ", lengths)
            print("lengths shape: ", len(lengths))
            print("actual_lengths: ", actual_lengths)
            print("actual_lengths shape: ", len(actual_lengths))
            optimizer.zero_grad()
            
            
            if use_gpu:
                inputs = inputs.to(device)# Move your inputs onto the gpu
                labels = labels.to(device) # Move your labels onto the gpu
            
                
#             outputs = model(inputs)
            enc_out = encoder(inputs)
            print('enc out shape: ', enc_out.size())
        
            temperature = 1
            test_pred = decoder.generate_caption(enc_out, maxSeqLen, temperature)
            for b in range(batch_size):
                flag = False
                for wi in range(maxSeqLen):
                    if test_pred[b, wi] == 2:
                        flag = True
                    if flag:
                        test_pred[b, wi] = 0
            print('test pred: ', test_pred)
            print('test pred shape: ', test_pred.size())
        
            outputs = decoder(labels, enc_out, actual_lengths)
            print('outputs shape: ', outputs.size())
            new_outputs = torch.zeros(batch_size, maxSeqLen, vocablen)
            for dim in range(maxSeqLen):
                for b in range(batch_size):
                    new_outputs[b, dim, 0] = 1.0
            new_outputs[:, :(outputs.shape[1]), :] = outputs
            new_outputs = new_outputs.permute(0, 2, 1).to(device)
            #del inputs
            loss = criterion(new_outputs, Variable(labels.long()))
            del labels
            del outputs

            loss.backward()
            loss = loss#.item()
            optimizer.step()

            if iter % 1 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss))

        
        # calculate val loss each epoch
#         val_loss, val_acc, val_iou = val(model, val_loader, criterion, use_gpu)
#         val_loss_set.append(val_loss)
#         val_acc_set.append(val_acc)
#         val_iou_set.append(val_iou)
        
#         print("epoch {}, time {}, train loss {}, val loss {}, val acc {}, val iou {}".format(epoch, time.time() - ts,
#                                                                                                loss, val_loss,
#                                                                                                val_acc,
#                                                                                                val_iou))        
        training_loss.append(loss)
        
        with open(logname, "a") as file:
            file.write("writing!\n")
            file.write("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
            file.write("\n training Loss:   " + str(loss.item()))
#             file.write("\n Validation Loss: " + str(val_loss_set[-1]))
#             file.write("\n Validation acc:  " + str(val_acc_set[-1]))
#             file.write("\n Validation iou:  " + str(val_iou_set[-1]) + "\n ")                                             
                                                                                                
                                                                                                
        
        # Early stopping
#         if val_loss < minLoss:
#             # Store new best
#             torch.save(model, name)
#             minLoss = val_loss#.item()
#             minLossIdx = epoch
            
        # If passed min threshold, and no new min has been reached for delta epochs
#         elif epoch > earliestStopEpoch and (epoch - minLossIdx) > earlyStopDelta:
#             print("Stopping early at {}".format(minLossIdx))
#             break
        

        
        
    with open(logname_summary, "a") as file:
            file.write("Summary!\n")
            file.write("Stopped early at {}".format(minLossIdx))
            file.write("\n training Loss:   " + str(training_loss))        
            file.write("\n Validation Loss: " + str(val_loss_set))
            file.write("\n Validation acc:  " + str(val_acc_set))
            file.write("\n Validation iou:  " + str(val_iou_set) + "\n ")
            
        
    #return val_loss_set, val_acc_set, val_iou_set

In [3]:
if __name__=='__main__':
    with open('TrainImageIds.csv', 'r') as f:
        reader = csv.reader(f)
        trainIds = list(reader)[0]
        
#     with open('TestImageIds.csv', 'r') as f:
#         reader = csv.reader(f)
#         testIds = list(reader)[0]
    
    trainIds = [int(i) for i in trainIds]
    #testIds = [int(i) for i in testIds[0]]
    
    # Will shuffle the trainIds incase of ordering in csv
    #random.shuffle(trainIds)
    #splitIdx = int(len(trainIds)/5)
    
    # Selecting 1/5 of training set as validation
    #valIds = trainIds[:splitIdx]
    #trainIds = trainIds[splitIdx:]
    #print(trainIds)
    
    
    trainValRoot = "./data/images/train/"
    #testRoot = "./data/images/test/"
    
    trainValJson = "./data/annotations/captions_train2014.json"
    #testJson = "./data/annotations/captions_val2014.json"
    
    
    with open('./data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    
    img_side_length = 700
    transform = tf.Compose([
        tf.Resize(img_side_length),
        #tf.RandomCrop(img_side_length),
        tf.CenterCrop(img_side_length),
        tf.ToTensor(),
    ])
    batch_size = 4
    shuffle = True
    num_workers = 1
    
    
    trainDl = get_loader(trainValRoot, trainValJson, trainIds, vocab, 
                         transform=transforms.ToTensor(), batch_size=batch_size, 
                         shuffle=False, num_workers=1)
#     valDl = get_loader(trainValRoot, trainValJson, valIds, vocab, 
#                          transform=None, batch_size=batch_size, 
#                          shuffle=shuffle, num_workers=1)
#    testDl = get_loader(testRoot, testJson, testIds, vocab, 
#                         transform=None, batch_size=batch_size, 
#                         shuffle=shuffle, num_workers=1)
    
    encoded_feature_dim = 56
    hidden_dim = 50
    
    encoder = Encoder(encoded_feature_dim)
    decoder = Decoder(encoded_feature_dim, hidden_dim, vocab.idx)
    
    criterion = nn.NLLLoss()
    
    epochs = 100
    trainEncoderDecoder(encoder, decoder, criterion, epochs, 
                        trainDl, trainDl, trainDl, "LSTM", batch_size, encoded_feature_dim, vocab.idx)
    
    
    
    
    
    
    

loading annotations into memory...
Done (t=0.91s)
creating index...
index created!
Loading results to logfile: ./logs/LSTM4.log
Loading Summary to : ./logs/LSTM_summary4.log


0it [00:00, ?it/s]

inp shape:  torch.Size([4, 3, 480, 640])
Labels
tensor([[  1,   4,  81,  14, 723, 225,  40,   4, 133,  80,  33, 279,  19,   2,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  1,   4,  81,  14, 455, 225,  40,   4, 133,  80,  33, 279,   2,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  1,   4, 331,  14,  52,  77,  40,   4, 265, 133,   2,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
 

1it [00:00,  2.42it/s]

epoch0, iter0, loss: -0.7143136262893677


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/datasets/home/home-02/64/364/rhadden/Image-Captioning-using-LSTM-network/data_loader.py", line 99, in collate_fn
    images = torch.stack(images, 0)
RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 480 and 500 in dimension 2 at /opt/conda/conda-bld/pytorch_1573049310284/work/aten/src/TH/generic/THTensor.cpp:689


In [ ]:
%debug

In [ ]:
print(vocab.word2idx['<start>'])
print(vocab.idx2word[0])

In [ ]:
trainIds.index(509365)

In [ ]:
print(trainIds[0])